# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [48]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, avg
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.window import Window

import pandas as pd
import numpy as np

spark = SparkSession \
    .builder \
    .appName("Wrangling Data") \
    .getOrCreate()

path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)

In [17]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



# Question 1

Which page did user id "" (empty string) NOT visit?

In [16]:
# Show visited pages for given user
pages_empty = (user_log.filter(user_log["userId"] == "")
               .select("page").
               dropDuplicates())
pages_empty.show()

+-----+
| page|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+



# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [22]:
# TODO: use this space to explore the behavior of the user with an empty string
(user_log.filter(user_log["userId"] == "")
               .select("ts", "registration", "level", "userAgent")
               .show()
)

+-------------+------------+-----+---------+
|           ts|registration|level|userAgent|
+-------------+------------+-----+---------+
|1513721196284|        null| free|     null|
|1513721274284|        null| paid|     null|
|1513722009284|        null| free|     null|
|1513723183284|        null| paid|     null|
|1513723184284|        null| paid|     null|
|1513723587284|        null| free|     null|
|1513724475284|        null| paid|     null|
|1513724530284|        null| paid|     null|
|1513728229284|        null| paid|     null|
|1513729051284|        null| paid|     null|
|1513729052284|        null| paid|     null|
|1513729376284|        null| paid|     null|
|1513729445284|        null| paid|     null|
|1513729762284|        null| paid|     null|
|1513729763284|        null| paid|     null|
|1513730128284|        null| free|     null|
|1513730129284|        null| free|     null|
|1513730621284|        null| free|     null|
|1513730622284|        null| free|     null|
|151373095

# Question 3

How many female users do we have in the data set?

In [29]:
(user_log.select('userId', 'gender')
         .dropDuplicates()
         .filter(user_log["gender"] == "F")
         .count()
#          .show()
)

462

# Question 4

How many songs were played from the most played artist?

In [43]:
user_log.filter(user_log.page == 'NextSong') \
       .select('Artist') \
       .groupBy('Artist') \
       .agg({'Artist':'count'}) \
       .withColumnRenamed('count(Artist)', 'Artistcount') \
       .sort(desc('Artistcount')) \
       .show(5)

+--------------------+-----------+
|              Artist|Artistcount|
+--------------------+-----------+
|            Coldplay|         83|
|       Kings Of Leon|         69|
|Florence + The Ma...|         52|
|            BjÃÂ¶rk|         46|
|       Dwight Yoakam|         45|
+--------------------+-----------+
only showing top 5 rows



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [49]:
# Filter out 0 sum and max sum to get more exact answer

function = udf(lambda ishome : int(ishome == 'Home'), IntegerType())

user_window = Window \
    .partitionBy('userID') \
    .orderBy(desc('ts')) \
    .rangeBetween(Window.unboundedPreceding, 0)

cusum = user_log.filter((user_log.page == 'NextSong') | (user_log.page == 'Home')) \
    .select('userID', 'page', 'ts') \
    .withColumn('homevisit', function(col('page'))) \
    .withColumn('period', Fsum('homevisit').over(user_window))

cusum.filter((cusum.page == 'NextSong')) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'}) \
    .agg({'count(period)':'avg'}).show()

+------------------+
|avg(count(period))|
+------------------+
| 6.898347107438017|
+------------------+

